# Crypto Market Making Bot for USD+/wETH and USD+/cbBTC

This notebook implements market making algorithms for Overnight.fi using Avellaneda-Stoikov methodology enhanced with Reinforcement Learning (RL).

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
import os
import importlib.util

# Configure plotting
plt.style.use('ggplot')
%matplotlib inline

# Add project root to path - use absolute path to ensure reliability
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to Python path")

In [ ]:
# Import project modules using a more robust approach
# Import the market_data module directly using importlib
market_data_path = os.path.join(project_root, 'src', 'utils', 'market_data.py')
print(f"Loading market_data.py from: {market_data_path}")

if os.path.exists(market_data_path):
    # Import the module using importlib
    spec = importlib.util.spec_from_file_location("market_data", market_data_path)
    market_data = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(market_data)
    
    # Get MarketDataHandler class from the module
    MarketDataHandler = getattr(market_data, 'MarketDataHandler')
    OnchainDataHandler = getattr(market_data, 'OnchainDataHandler')
    
    print(f"Successfully imported MarketDataHandler")
else:
    print(f"Error: market_data.py not found at {market_data_path}")

# Import other project modules
from src.models.avellaneda_stoikov import AvellanedaStoikovModel
from src.models.rl_enhanced_model import RLEnhancedModel
from src.backtesting.backtest_engine import BacktestEngine
from src.data.data_processor import DataProcessor

## 1. Generate Market Data

In [ ]:
# Generate synthetic market data
data_processor = DataProcessor()

# Define date range
start_date = datetime.now() - timedelta(days=30)
end_date = datetime.now()
timestamps = pd.date_range(start=start_date, end=end_date, freq='1min')

# Generate price data
np.random.seed(42)
eth_price = 2000.0
btc_price = 50000.0

eth_prices = [eth_price]
btc_prices = [btc_price]

for _ in range(len(timestamps) - 1):
    eth_return = np.random.normal(0, 0.001)
    btc_return = np.random.normal(0, 0.001)
    
    eth_price *= (1 + eth_return)
    btc_price *= (1 + btc_return)
    
    eth_prices.append(eth_price)
    btc_prices.append(btc_price)

# Create DataFrames
eth_usdt_data = pd.DataFrame({
    'open': eth_prices,
    'high': [p * (1 + np.random.uniform(0, 0.002)) for p in eth_prices],
    'low': [p * (1 - np.random.uniform(0, 0.002)) for p in eth_prices],
    'close': eth_prices,
    'volume': np.random.uniform(10, 100, size=len(timestamps))
}, index=timestamps)

# Add technical features
eth_usdt_data = data_processor.add_technical_features(eth_usdt_data)

# Display data
eth_usdt_data.head()

## 2. Implement Avellaneda-Stoikov Market Making Model

In [ ]:
# Initialize the market making model
market_maker = AvellanedaStoikovModel(
    risk_aversion=1.0,  # Risk aversion parameter
    time_horizon=1.0,   # Time horizon in days
    volatility=eth_usdt_data['volatility'].mean()  # Use mean volatility from data
)

# Calculate quotes for a specific time
sample_time = eth_usdt_data.index[100]
sample_data = eth_usdt_data.loc[sample_time]
mid_price = sample_data['mid_price']

# Set model parameters
market_maker.set_parameters(volatility=sample_data['volatility'])
market_maker.update_inventory(0)  # Start with zero inventory

# Calculate optimal quotes
bid_price, ask_price = market_maker.calculate_optimal_quotes(mid_price)

print(f"Mid Price: ${mid_price:.2f}")
print(f"Bid Price: ${bid_price:.2f}")
print(f"Ask Price: ${ask_price:.2f}")
print(f"Spread: ${ask_price - bid_price:.2f} ({(ask_price - bid_price) / mid_price * 100:.4f}%)")

## 3. Backtest the Market Making Strategy

In [ ]:
# Initialize backtest engine
backtest_engine = BacktestEngine(
    market_data=eth_usdt_data,
    initial_capital=10000.0,
    transaction_fee=0.001
)

# Run backtest with the market making model
backtest_results = backtest_engine.run_backtest(
    model=market_maker,
    params={
        'risk_aversion': 1.0,
        'time_horizon': 1.0
    },
    max_inventory=50,
    volatility_window=20
)

# Display performance metrics
print("Performance Metrics:")
for key, value in backtest_results['metrics'].items():
    print(f"{key}: {value}")

## 4. Visualize Backtest Results

In [ ]:
# Extract performance data
if backtest_results['positions'] is not None and not backtest_results['positions'].empty:
    performance_data = backtest_results['positions']
    
    # Plot equity curve
    plt.figure(figsize=(12, 6))
    plt.plot(performance_data['timestamp'], performance_data['total_value'], label='Portfolio Value')
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Time')
    plt.ylabel('Value ($)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Plot inventory
    plt.figure(figsize=(12, 6))
    plt.plot(performance_data['timestamp'], performance_data['inventory'], label='Inventory')
    plt.title('Inventory Over Time')
    plt.xlabel('Time')
    plt.ylabel('Inventory')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No position data available for plotting")

## 5. Enhance with Reinforcement Learning

In [ ]:
# Initialize the RL-enhanced model
rl_model = RLEnhancedModel(base_model=market_maker)

# Run backtest with the enhanced model
rl_backtest_results = backtest_engine.run_backtest(
    model=rl_model,
    max_inventory=50,
    volatility_window=20
)

# Compare performance
if backtest_results['positions'] is not None and rl_backtest_results['positions'] is not None:
    base_performance = backtest_results['metrics']['total_pnl']
    rl_performance = rl_backtest_results['metrics']['total_pnl']
    
    print(f"Base Model PnL: ${base_performance:.2f}")
    print(f"RL-Enhanced Model PnL: ${rl_performance:.2f}")
    print(f"Improvement: {(rl_performance - base_performance) / abs(base_performance) * 100:.2f}%")
else:
    print("Insufficient data for comparison")